In [1]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import warnings
warnings.filterwarnings('ignore')
import xml.etree.ElementTree as ET
from ast import literal_eval

In [2]:
TRAIN_DATA_PATH = r"C:\Users\USER\Documents\DataScience\Skripsiiuu\Aspect-Term-Extraction\Dataset\train.xml"
TEST_DATA_PATH = r"C:\Users\USER\Documents\DataScience\Skripsiiuu\Aspect-Term-Extraction\Dataset\test.xml"
PRE_PROC_TRAIN_PATH = "pre_processed.csv"
PRE_PROC_TEST_PATH = "pre_processed_test.csv"

In [3]:
import pandas as pd

class IOB:
    def convert_into_bio(self, text, predicted_aspect_terms_list, common_words_list):
        text_list = text.split()
        # print(text_list)
        df = pd.DataFrame({'text': text_list, 'BIO': 'O'})

        bio_dict = self.bio_mapper(common_words_list=common_words_list, prediction=predicted_aspect_terms_list)
        bio_filtered_dict = self.filter_predicted_words(bio_dict)

        for item in bio_filtered_dict:
            final_list = self.matcher(text, item)
            if final_list[1] - final_list[0] == 0:
                df.at[final_list[0], 'BIO'] = 'B'
            else:
                df.at[final_list[0], 'BIO'] = 'B'
                for index in range(final_list[0] + 1, final_list[1] + 1):
                        df.at[index, 'BIO'] = 'I'

        return df

    def bio_mapper(self, common_words_list, prediction):
        bio_dict = dict()

        for i in range(len(common_words_list)):
            bio_dict[common_words_list[i]] = prediction[i]

        return bio_dict

    def matcher(self, text, word):
        split_word = word.split()
        split_text = text.split(word)
        start=0
        end=0
        if len(split_text) > 1:
            start = len(split_text[0].split())
            end = start + len(split_word) - 1
        return start, end

    def filter_predicted_words(self, _dict):

        final_list = []

        for key, value in _dict.items():
            if value == 1:
                final_list.append(key)

        return final_list



In [22]:
import xml.etree.ElementTree as ET
import pandas as pd


def preprocess(path):
    tree=ET.parse(path)
    root = tree.getroot()
    text_list = []
    opinion_list = []
    for review in root.findall('Review'):
        text_string=""
        opinion_inner_list=[]
        for sent in review.findall('./sentences/sentence'):
            text_string= text_string+ " "+ sent.find('text').text.lower()
        text_list.append(text_string)
        for opinion in review.findall('./sentences/sentence/Opinions/Opinion'):
            opinion_dict = {
                opinion.get('target'):opinion.get('polarity')
            }
            opinion_inner_list.append(opinion_dict)
        opinion_list.append(opinion_inner_list)
    return text_list,opinion_list

In [23]:
preprocess(TRAIN_DATA_PATH)

([' judging from previous posts this used to be a good place, but not any longer. we, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude. they never brought us complimentary noodles, ignored repeated requests for sugar, and threw our dishes on the table. the food was lousy - too sweet or too salty and the portions tiny. after all that, they complained to me about the small tip. avoid this place!',
  ' i have eaten at saul, many times, the food is always consistently, outrageously good. saul is the best restaurant on smith street and in brooklyn. the duck confit is always amazing and the foie gras terrine with figs was out of this world. the wine list is interesting and has many good values. for the price, you cannot eat this well in manhattan.',
  ' i was very disappointed with this restaurant. ive asked a cart attendant for a lotus leaf wrapped rice and she replied back rice and just walked away. i h

In [38]:
#Selecting only 20 most common aspect.
def get_most_common_aspect(aspect_list):
    import nltk
    aspect_terms = []

    for inner_list in aspect_list:
        if inner_list is not None:
            for _dict in inner_list:
                for key in _dict:
                    aspect_terms.append(key)

    most_common_aspect = [k for k, v in nltk.FreqDist(aspect_terms).most_common(1000)]
    return most_common_aspect
     

In [49]:
def get_data_frame(text_list, opinion_list, most_common_aspect):
    data = {'Review': text_list}
    df = pd.DataFrame(data)
    if opinion_list:
        for inner_list in opinion_list:
            for _dict in inner_list:
                for key in _dict:
                    if key in most_common_aspect:
                        df.loc[opinion_list.index(inner_list),key]=_dict[key]
    return df

In [40]:
def get_aspect_data_frame(df, most_common_aspect):
    for common_aspect in most_common_aspect:
        df[common_aspect] = df[common_aspect].replace(['positive','negative','neutral','conflict'],[1,1,1,1])
    df = df.fillna(0)
    return df

In [41]:
train_text_list,train_opinion_list = preprocess(TRAIN_DATA_PATH)
most_common_aspect = get_most_common_aspect(train_opinion_list)

In [43]:
print(most_common_aspect)

['NULL', 'food', 'place', 'service', 'restaurant', 'staff', 'Service', 'atmosphere', 'pizza', 'sushi', 'decor', 'waiter', 'Food', 'menu', 'dishes', 'waitress', 'meal', 'drinks', 'wine list', 'spot', 'ambience', 'rolls', 'dessert', 'fish', 'portions', 'bagels', 'manager', 'wine', 'Pizza', 'ingredients', 'waiters', 'location', 'Indian food', 'view', 'crust', 'hostess', 'wait', 'people', 'toppings', 'wait staff', 'rice', 'dinner', 'lobster sandwich', 'Sushi', 'Casa La Femme', 'meals', 'seafood', 'vibe', 'setting', 'wines', 'server', 'salad', 'chef', 'ambiance', 'servers', 'sandwiches', 'PLACE', 'customer service', 'music', 'salmon', 'Suan', 'chicken', 'dish', 'The Four Seasons', 'hot dog', 'hot dogs', 'Fish', 'Prix Fixe menu', 'portion', 'waitstaff', 'Leon', 'specials', 'sashimi', 'Mizu', 'bar', 'tuna', 'bagel', 'salads', 'spicy Tuna roll', 'Delivery', 'resturant', 'feel', 'owner', 'Decor', 'Cheese plate', 'calamari', 'dim sum', 'sauce', 'martinis', 'Thai food', 'entree', 'Taiwanese food'

In [50]:
df_train = get_data_frame(train_text_list,train_opinion_list, most_common_aspect)

In [51]:
df_train

,Review,place,staff,NULL,food,portions,Saul,foie gras terrine with figs,duck confit,wine list,...,bottle of wine,frontman,Mussles,Lamb special,flank steak,The Four Seasons restaurant,fish tacos,fire place,Casa la Femme,pasta
0,"judging from previous posts this used to be a good place, but not any longer. we, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude. they never brought us complimentary noodles, ignored repeated requests for sugar, and threw our dishes on the table. the food was lousy - too sweet or too salty and the portions tiny. after all that, they complained to me about the small tip. avoid this place!",negative,negative,negative,negative,negative,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"i have eaten at saul, many times, the food is always consistently, outrageously good. saul is the best restaurant on smith street and in brooklyn. the duck confit is always amazing and the foie gras terrine with figs was out of this world. the wine list is interesting and has many good values. for the price, you cannot eat this well in manhattan.",NaN,NaN,positive,positive,NaN,positive,positive,positive,positive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"i was very disappointed with this restaurant. ive asked a cart attendant for a lotus leaf wrapped rice and she replied back rice and just walked away. i had to ask her three times before she finally came back with the dish ive requested. food was okay, nothing great. chow fun was dry; pork shu mai was more than usually greasy and had to share a table with loud and rude family. i/we will never go back to this place again.",negative,NaN,negative,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"went on a 3 day oyster binge, with fish bringing up the closing, and i am so glad this was the place it o trip ended, because it was so great! service was devine, oysters where a sensual as they come, and the price can't be beat!!! you can't go wrong here.",NaN,NaN,positive,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"every time in new york i make it a point to visit restaurant saul on smith street. everything is always cooked to perfection, the service is excellent, the decor cool and understated. i had the duck breast special on my last visit and it was incredible. can't wait wait for my next visit.",NaN,NaN,positive,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,"poor service and management don’t go to this place! had an awful experience at casa la femme on a saturday dinner. appetizers took nearly an hour. when the main course finally arrived (another 45mins) half of our order was missing. never got an explanation as to what was going on. the manager was rude and handled the situation extremely poorly. when we threatened to leave, we were offered a meager discount even though half the order was missing. on the way out, we heard of other guests complaining about similar issues. can’t believe how an expensive nyc restaurant can be so disrespectful to its clients.",negative,NaN,negative,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,negative,NaN
346,"what a hassle! the food is very good, but not outstanding. there is no way it justifies the accolades it receives, the attitude of the staff or the wait for a table. on our last visit, they skipped over our name on the list, leaving us waiting an extra hour for a table. mistakes happen, but they are usually accompanied by an apology, perhaps even a glass of wine...but not the grunt that we received from the al di la staff. without the wait or attitude, i would probably go back, but there are just too many other restaurants in brooklyn that are as good or better and don't make me jump through hoops.",NaN,negative,negative,neutral,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,N

In [53]:
df_train_aspect = get_aspect_data_frame(df_train, most_common_aspect)
df_train_aspect.head(10)

,Review,place,staff,NULL,food,portions,Saul,foie gras terrine with figs,duck confit,wine list,...,bottle of wine,frontman,Mussles,Lamb special,flank steak,The Four Seasons restaurant,fish tacos,fire place,Casa la Femme,pasta
0,"judging from previous posts this used to be a good place, but not any longer. we, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude. they never brought us complimentary noodles, ignored repeated requests for sugar, and threw our dishes on the table. the food was lousy - too sweet or too salty and the portions tiny. after all that, they complained to me about the small tip. avoid this place!",1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"i have eaten at saul, many times, the food is always consistently, outrageously good. saul is the best restaurant on smith street and in brooklyn. the duck confit is always amazing and the foie gras terrine with figs was out of this world. the wine list is interesting and has many good values. for the price, you cannot eat this well in manhattan.",0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"i was very disappointed with this restaurant. ive asked a cart attendant for a lotus leaf wrapped rice and she replied back rice and just walked away. i had to ask her three times before she finally came back with the dish ive requested. food was okay, nothing great. chow fun was dry; pork shu mai was more than usually greasy and had to share a table with loud and rude family. i/we will never go back to this place again.",1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"went on a 3 day oyster binge, with fish bringing up the closing, and i am so glad this was the place it o trip ended, because it was so great! service was devine, oysters where a sensual as they come, and the price can't be beat!!! you can't go wrong here.",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"every time in new york i make it a point to visit restaurant saul on smith street. everything is always cooked to perfection, the service is excellent, the decor cool and understated. i had the duck breast special on my last visit and it was incredible. can't wait wait for my next visit.",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,"i had my eyes on this place, promising myself i will one day 'give it a try.' and i hate to say this but i doubt i'll ever go back. the food is very average...the thai fusion stuff is a bit too sweet, every thing they serve is too sweet here. the only thing i moderately enjoyed was their grilled chicken special with edamame puree. i had never had edamame pureed before but i thought it was innovative and tasty (could've used a bit more salt). anyways, if you're in the neighborhood to eat good food, i wouldn't waste my time trying to find something, rather go across the street to tamari. the decor is night tho...but they really need to clean that vent in the ceiling...its quite un-appetizing, and kills your effort to make this place look sleek and modern.",1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,"we ate outside at haru's sake bar because haru's restaurant next door was overflowing. what's the difference between the two? their sake list was extensive, but we were looking for purple haze, which wasn't listed but made for us upon request! the spicy tuna roll was unusually good and the rock shrimp tempura was awesome, great appetizer to share! we went around 9:30 on a friday and it had died down a bit by then so the service was great!",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,"we love th pink pony. the perfect spot. food-awesome. service- friendly and attentive. ambiance- relaxed and stylish. don't judge this place prima facie, you have to try it to 